## Introduction
This notebook shows how the different parts fit together. Each part is discussed in more detail in separate notebooks referenced below

## Solution overview
The solution consists of 5 processing steps, each individually maintainable and replacable:
| Step | Name      | Brief description                                                  | Inputs                                                         | Outputs                                         | Major dependencies |
| ---- | --------- | ----------------------------------------------------------------   | -------------------------------------------------------------  | ----------------------------------------------- | --------------------------------------------------- |                     
|    1 | Tracing   | Extracts model structure and node metadata (parameter counts, FLOPS, activations) | PyTorch model object<br>Input tensor sizes      | List of nodes<br>List of connections            | [torch.fx](https://pytorch.org/docs/stable/fx.html)<br>[torchprofile](https://github.com/zhijian-liu/torchprofile) |
|    2 | Merging   | Prevents small operations (e.g. ReLU) from taking up a full node   | List of nodes<br>List of connections                           | Populates `is_subnode` field for each node      |                        
|    3 | Layout    | Determines the x- and y-coordinates at which to place each node    | List of nodes<br>List of connections                           | Populates `x` and `y` coordinates for each node | [munkres](https://github.com/bmc/munkres/)          |
|    4 | Coloring  | Assign colors to nodes based on the operation they perform         | List of nodes with `operation` field populated                 | Populates `op_color` field for each node        | [plotly.colors](https://plotly.com/python/discrete-color) |
|    5 | Rendering | Plots the nodes and defines user interactions                      | List of nodes with all fields populated<br>List of connections | Portable figure<br>Interactive widget           | [plotly](https://plotly.com/python/)<br>[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) |


## Storyline and notebook references
* The **Layout** algorithm was developed first in [01_explore_layout.ipynb](./01_explore_layout.ipynb).
  - At this point, none of the other components existed, so two simple viewers were developed in the notebook itself to visualize results.
  - The best of these was later refactored for reusability and placed as a standalone class `ArcViewer` in [static_viewers.py](../idlmav/static_viewers.py) 
  - For the same reason, some random graph generation functions were developed to provide inputs to the layout algorithm
  - The best of these was later refactored for reusability and placed as a standalone function `create_random_sample_graph` in [layout.py](../idlmav/layout.py)
  - These reuable components were used during development of some of the other components, but their use gradually faded as better components were developed
  - The final layout algorithm was polished and maintained in the `MavLayout` class in [layout.py](../idlmav/layout.py), which is exported via the `layout_graph_nodes` function in the same file
  - The notebook [02_test_layout.ipynb](./02_test_layout.ipynb) runs this maintained version of the algorithm to confirm that it is working
* The **Rendering** algorithm was developed next in [03_explore_rendering.ipynb](./03_explore_rendering.ipynb)
  - During this experiment, two more detailed experiments were performed in [04_explore_plotly_interaction.ipynb](./04_explore_plotly_interaction.ipynb) and [05_explore_js_interaction.ipynb](./05_explore_js_interaction.ipynb) to try out different approaches to interactivity
  - No ideal approach was found that would suit all use cases. Therefore, two approaches were settled on to cater for different use cases
    - The more interactive approach was implemented in [widget_renderer.py](../idlmav/renderers/widget_renderer.py) and provides the most interactivity, but requires a running back-end or kernel to service JS requests.
    - The more portable approach was implemented in [figure_renderer.py](../idlmav/renderers/figure_renderer.py) and creates an output that works well even when browsed on [nbviewer](https://nbviewer.org/) without a running back-end
  - The notebook [06_test_rendering.ipynb](./06_test_rendering.ipynb) runs the maintained versions of the rendering algorithms to confirm that they are working
* The **Tracing** algorithm was developed next in [07_explore_tracing.ipynb](./07_explore_tracing.ipynb)
  - During this experiment, a [tutorial by James Reed](https://github.com/pytorch/tutorials/pull/1319) was copied to [08_fx_profiling_tutorial.ipynb](./08_fx_profiling_tutorial.ipynb) and executed from there to get a better understanding of `torch.fx` and its `Interpreter` class
  - The `Interpreter` class from `torch.fx` was experimented with in more detail in [09_explore_fx_interpreter.ipynb](./09_explore_fx_interpreter.ipynb)
  - A strategy for tracing was documented in [07_explore_tracing.ipynb](./07_explore_tracing.ipynb), implemented in [tracing.py](../idlmav/tracing.py) and tested in [10_test_tracing.ipynb](./10_test_tracing.ipynb)
* The **Coloring** and **Merging** algorithms were implemented last
  - These algorithms were simpler and required less iterative experimentation.
  - At this point, the IDLMAV solution was working end-to-end, just without coloring and merging.
  - For these reasons, they were implemented directly in [coloring.py](../idlmav/coloring.py) and [merging.py](../idlmav/merging.py), respectively
  - Some experiments on how to interact with `plotly.colors` were performed in [11_explore_colors.ipynb](./11_explore_colors.ipynb)
  - Testing was performed in [12_test_idlmav.ipynb](./12_test_idlmav.ipynb) using the `reload_imports()` function to reload updates to the ".py" files without restarting the notebook or kernel